In [1]:
#import libraries
import json


In [2]:
#scrape all initial websites to train the module
with open("../dbHandler/iab_categories_initial_data.json", "r") as jsonobj:
    initial_data_json = json.load(jsonobj)

website_array = []
websites_to_train_content = {}

for item in initial_data_json["iabCategories"].values():
    website_array.extend(item["websites"])

total_websites = len(website_array)
website_loading_index = 0

for website in website_array:
    print(f"loading {website_loading_index}/{total_websites} ======> {website}")
    website_loading_index = website_loading_index + 1

    with open("../scrapeContent/website_to_scrape.txt", "w") as file:
        file.write(website)

    %run "../scrapeContent/scrapeContent.ipynb"
    #%run "../dbHandler/addNewLanguageKeywords.ipynb" #the initial websites will be all in english, if not the keywords should all be grabbed from the database after running this line
    
    with open("../scrapeContent/scraped_content_output.json", "r") as scrapedContentJsonObj:
        scraped_content_output = json.load(scrapedContentJsonObj)

    #create new json where all the websites are present
    websites_to_train_content[website] = {"description":scraped_content_output["description"], "alt_images_texts": scraped_content_output["alt_images_texts"], "content": scraped_content_output["content"],"status_code":scraped_content_output["status_code"]}

print(f"loading {website_loading_index}/{total_websites}")
#this is only if we want to see the content of the json above
#with open("../scrapeContent/websites_to_train_content.json", "w") as outputFile:
#    json.dump(websites_to_train_content, outputFile)


In [3]:
#process the json above, into trainig data, filtering it with all keywords 
processed_website_content = {}

for website, contents in websites_to_train_content.items():
    keywords_array = []

    #grab keywords and the website category_name
    for item in initial_data_json["iabCategories"].values():
        keywords_array.extend(item["keywords"])

        if website in item["websites"]:
            category_name = item["name"]

     #grab main_title assuming the first element is always the title in the head of the page
    main_title = contents.get("content", [])[0]["text"] if contents.get("content") else ""
    
    #initiate processed_website_content json
    processed_website_content[website] = {
        "status_code": contents.get("status_code", ""),
        "category_name": category_name,
        "head_title": main_title,
        "description": contents.get("description", ""),
        "alt_images_texts": ""  
    }

    #store all alt_image_text s in an array
    alt_images_texts_array = contents.get("alt_images_texts", "")

    #filter the text with the keywords and store all together in a string based on the elements there are in
    for content in contents.get("content", [])[1:]:
        element = content["element"]

        
        if element not in processed_website_content[website]:
            processed_website_content[website][element] = ""
        processed_website_content[website][element] += content["text"] + " "

    #filter alt_images_texts_array values with the keywords and store all in a string
    for alt_text in alt_images_texts_array:
        
        processed_website_content[website]["alt_images_texts"] += alt_text + " "

In [4]:
#store the json in a file
with open("../scrapeContent/processed_websites_content.json", "w") as outputFile:
    json.dump(processed_website_content, outputFile)